# Декораторы

Людей много, и они все разные. Как бы это ни звучало, но общество требует (обычно от государства) с такими разными людьми делать, тем не менее, одинаковые стандартные вещи. Например, регистрировать людей в фискальных органах, платить пенсионерам пенсию, делать детям прививки от кори. За отдельными исключениями, это стандартное действие.

Функции тоже все разные. Стандартная операция — вызвать функцию. Ещё её можно использовать как значение, чтобы куда-нибудь передать. Удобная операция (уже требующая использовать функцию, как значение) — декорирование. Перед тем, как получить своё имя, функцию «пропускают» через другую функцию.

Например, мы хотим, чтобы функция перед вызовом печатала свой параметр (для простоты будем считать, что он один), а после выхода — возвращаемое значение. Так тому и быть.

## Пример

In [1]:
def test_function(n):
    return n * 2

def logged(genuine_function):  # Эта функция реализует декоратор. Она получает на вход исходную (настоящую) функцию
    def fake_function(argument):  # Создаёт внутри фальшивую функцию
        result = genuine_function(argument) # Считает значение настоящей функции
        print("Функция:", repr(genuine_function), ", аргумент:", argument, ", значение:", result)  # Печатает информацию
        return result  # И возвращает значение
    return fake_function  # Ещё не забыли? Декоратор получает функцию и возвращает тоже функцию

test_function = logged(test_function)  # Пропускаем test_function через декоратор и запоминаем под тем же именем

print(test_function(5))

Функция: <function test_function at 0x000001463F4D1400> , аргумент: 5 , значение: 10
10


Можно и поизящнее — для этого есть специальный удобный синтаксис:

In [2]:
@logged
def test_function_2(n):
    return n * 3

print(test_function(3))

Функция: <function test_function at 0x000001463F4D1400> , аргумент: 3 , значение: 6
6


## А зачем вообще всё это нужно?

* Декораторы `@staticmethod` и `@classmethod`, которые позволяют вызывать статический метод от экземпляра объекта (удобно!)
* Регистрация функций, как обработчиков событий, например, в веб-приложениях (пример с фреймфорком Flask):

```
rom flask import Flask
app = Flask(__name__)

@app.route('/hello')
def hello_world():
    return 'Hello, World!'
```

После этого по запросу `http://сервер/hello` браузер получит обратно строку "Hello, World!". Т.е. декоратор саму функцию не «переделывает», зато сообщает фреймворку, что она понадобится в такой-то ситуации.

* Изменение свойств функций, как в нашем примере

Стоп. А мы реально хотим, чтобы прямо на консоль (точнее в `STDOUT`) выдавался весь наш лог — кого с какими параметрами вызвали?.. Так себе решение. Давайте сделаем декоратор, который позволит писать в произвольный поток. В нашем примере будем писать в `STDERR`, но это легко переделать под запись в файл на диске, например. Приступим.

In [3]:
import sys

def stream_logged(stream: 'file'):
    """Log function arguments and value to stream"""
    # Эта функция — не декоратор. Получив аргумент, она создаст и вернёт декоратор!

    # А дальше — бессовестный копипаст сверху
    def logging_decorator(genuine_function):  # Эта функция реализует декоратор. Она получает на вход исходную (настоящую) функцию
        def fake_function(argument):  # Создаёт внутри фальшивую функцию
            result = genuine_function(argument) # Считает значение настоящей функции
            print("Функция:", repr(genuine_function), ", аргумент:", argument, ", значение:", result, file=stream)  # Печатает информацию
            return result  # И возвращает значение
        return fake_function  # Ещё не забыли? Декоратор получает функцию и возвращает тоже функцию

    return logging_decorator # Ну и наконец выдайм сам декоратор

# Чтобы получить декоратор, мы вызываем stream_logged от параметра —
# файлового объекта. Полученный декоратор уже применяется к функции.
# А для «потребителя» выглядит просто как декоратор с параметром...
@stream_logged(sys.stderr) 
def rev(s):
    return list(s) == list(reversed(s))

print(rev("арозаупаланалапуазора"))

True


Функция: <function rev at 0x000001463F4D1D08> , аргумент: арозаупаланалапуазора , значение: True


То, что оно красненькое — это не ошибка. Это последствие нашего желания печатать в `STDERR`.

## Задание

Ну и как без него?

Сделать «декоратор с параметром» (но мы-то знаем), который будет применять функцию к арументу заданное количество раз. Т.е. такой:

```
def repeat(n):  # Вот это и надо реализовать
    ...
    ...


@repeat(2)
def plus_1(x):
    return x + 1


@repeat(0)
def mul_2(x):
    return x * 2

print(plus_1(3))  # должно выдать 5
print(mul_2(4))  # должно выдать 4
```